In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm, trange
from matplotlib import pyplot as plt

from micro_price_trading.config import DATA_PATH

In [2]:
with open('simulations.npy', 'rb') as file:
    input_data = np.load(file)

In [8]:
def CI(array, confidence = .95):
    half_width = 1.96*array.std()/np.sqrt(len(array))
    mu = array.mean()
    return mu - half_width, mu, mu + half_width

In [4]:
q_values = pd.read_csv(DATA_PATH.joinpath('q_values_9_27.csv'), index_col=0)

In [5]:
def optimal_twap(t, threshold=0.15):
    
    intervals = [range(idx*5, (idx+1)*5) for idx in range(input_data.shape[1]//t)]
    asset1_buy_prices = list()
    asset2_buy_prices = list()
    data_copy = input_data[:, :-1, :].copy()
    
    for sim in tqdm(data_copy):
        a1_buys = list()
        a2_buys = list()
        for interval in intervals:
            subset = sim[interval, :]
            for idx, entry in enumerate(subset):
                q_vals = q_values.iloc[int(entry[0])].copy()
                choice = np.argmax(q_vals)
                if choice == 0 and q_vals[0]-q_vals[1] > threshold:
                    a1_buys.extend([True]+[False]*(t-idx-1))
                    a2_buys.extend([False]*(t-idx))
                    break
                elif choice == 1 and q_vals[1]-q_vals[0] > threshold:
                    a1_buys.extend([False]*(t-idx))
                    a2_buys.extend([True]+[False]*(t-idx-1))
                    break
                else:
                    a1_buys.append(False)
                    a2_buys.append(False)
            else:
                if a1_buys[-1] + a2_buys[-1] < 1:
                    choice = np.argmax(q_values.iloc[int(entry[0]), 1:])
                    if choice == 0:
                        a1_buys[-1] = True
                    else:
                        a2_buys[-1] = True
        asset1_buy_prices.append(sim[a1_buys, 1].mean())
        asset2_buy_prices.append(sim[a2_buys, 2].mean())
    return np.array(asset1_buy_prices), np.array(asset2_buy_prices)

In [6]:
a1, a2 = optimal_twap(5)

In [9]:
CI(a1), CI(a2)

((18.311837368010096, 18.314020058404846, 18.316202748799597),
 (16.660502427064944, 16.661646826952055, 16.662791226839165))